In [184]:
from PreProcessingBASD_v2 import *
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output
from sklearn.metrics import auc , roc_auc_score
from sklearn.model_selection import train_test_split
sys.path.append("/home/advice/Python/SR/Custom/")
from RAdam import RAdamOptimizer
import seaborn as sns
import re , os
from ColumnMatch import MatchVariable
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [185]:
ResultPath = "./Result/Ensemble"
weightResultPath = os.path.join(ResultPath, "weight")
testPath = os.path.join(ResultPath, "test")
if tf.io.gfile.exists(ResultPath) :
    tf.io.gfile.rmtree(ResultPath)
    tf.io.gfile.makedirs(ResultPath)
    tf.io.gfile.makedirs(weightResultPath)
    tf.io.gfile.makedirs(testPath)
else :
    tf.io.gfile.makedirs(ResultPath)
    tf.io.gfile.makedirs(weightResultPath)
    tf.io.gfile.makedirs(testPath)

In [186]:
data = pd.read_csv("./../../../Data/kdd/uci/uci_creditcard-train-0.2-0.0.csv")
in_var = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6",
          "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
          "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]
target_var = ['default payment next month']
fac_var = [ 'SEX','EDUCATION','MARRIAGE',]
num_var = [i for i in in_var if not i in fac_var]
#in_var = num_var + fac_var
data[fac_var] = data[fac_var].astype("str")

In [187]:
cat_col = data.select_dtypes("object").columns.tolist()
cat_col
#onehot_data = pd.get_dummies(data , columns= cat_col)

['SEX', 'EDUCATION', 'MARRIAGE']

In [188]:
select_var = in_var + target_var + ["sep_idx"]
data = data[select_var]

In [189]:
train = data[data.sep_idx ==1 ]
valid = data[data.sep_idx ==0 ]
## 
train = data
# _ = train.pop("sep_idx")
# _ = valid.pop("sep_idx")

In [190]:
test = pd.read_csv("./../../../Data/kdd/uci/uci_creditcard-test-0.2-0.0.csv")
test[fac_var] = test[fac_var].astype("str")
# test = test.drop(["ID","sep_idx"],axis=1)
# test.reset_index(drop=True ,inplace=True)

In [191]:
train.shape, valid.shape, test.shape

((30000, 25), (8982, 25), (6004, 26))

In [192]:
trans_info = {}
Steps = [
    ("NumericImpute", MissingHandling(method="mean", 
                               trans_info= trans_info,
                               var= num_var)) , 
    ("FactorImpute", MissingHandling(method="most_frequent", 
                               trans_info= trans_info,
                               var= fac_var)) , 
    ("numeric", NumericHandler(method="normal",
                               trans_info= trans_info,
                               num_var=num_var)),
]

pipe = Pipeline(Steps)
pipe.fit(train)

Transform MissingData Method : mean
Transform MissingData Method : most_frequent


Pipeline(memory=None,
         steps=[('NumericImpute',
                 MissingHandling(method=None,
                                 trans_info={'fill_value': array([ 1.67484323e+05,  3.54752228e+01, -1.70900886e-02, -1.25958973e-01,
       -1.65824567e-01, -2.16004663e-01, -2.64603095e-01, -2.86467414e-01,
        5.10673010e+04,  4.95652237e+04,  4.70681142e+04,  4.32569520e+04,
        4.04164970e+04,  3.88574464e+04,  5.69172658e+03,  5.81409563e+03...
        5.13448505e+03,  4.83411333e+03,  4.76632123e+03,  5.24146976e+03]),
                                            'norm_std': array([1.29747662e+05, 8.24146160e+00, 9.99950166e-01, 1.07116176e+00,
       1.07345534e+00, 1.04935645e+00, 1.01015089e+00, 1.03371751e+00,
       6.54338544e+04, 6.39443098e+04, 6.25215503e+04, 5.72229826e+04,
       5.45746221e+04, 5.33084779e+04, 1.51993804e+04, 1.80845825e+04,
       1.44245395e+04, 1.38536166e+04, 1.33906631e+04, 1.61330268e+04])}))],
         verbose=False)

In [193]:
pipe_op = BADSPipeLine("./0405_Piepeline.pkl")
pipe_op.save(pipe)
pipe_op.load()
pipe_op.transform(test)

train = pipe_op.transform(train)
valid = pipe_op.transform(valid)
test = pipe_op.transform(test)

Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData
Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData
Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData
Transform MissingData Method : mean
Transform MissingData Method : most_frequent
Transform NumericData


In [194]:
train[fac_var].dtypes

SEX          object
EDUCATION    object
MARRIAGE     object
dtype: object

In [195]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
category_info = {}

class CategroyLabelEncoding(object) :
    def __init__(self , fac_var , in_var) :
        self.fac_var = fac_var
        self.in_var = in_var
        self.category_info = {}
    
    def fit(self, data) :
        for feat in self.fac_var:
            findIDX = [idx for idx, i in enumerate(self.in_var) if i == feat][0]
            lbe = LabelEncoder()
            lbe.fit(data[feat].values)
            diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_) + 1))
            self.category_info[feat] = diz_map_train
        return "label encoding"
    
    def transform(self,data) :
        for feat in self.fac_var:
            findIDX = [idx for idx, i in enumerate(self.in_var) if i == feat][0]
            current_key = list(self.category_info[feat].keys())
            remain_set = set(data[feat]).difference(current_key)
            if remain_set == set() :
                pass
            else :
                for i in remain_set :  ## if not in train impose 0
                    self.category_info[feat].update({i : 0})
#         data[feat] = [diz_map_train[i] for i in data[feat].values]
        print(self.category_info)
        data2 = data.replace(self.category_info)
        for key, value in  self.category_info.items() :
            data2[key] = pd.Categorical(data2[key],list(value.values()))
        return data2

    
testclass = CategroyLabelEncoding(fac_var , in_var)
testclass.fit(train)
train = testclass.transform(train)

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}


In [196]:
valid = testclass.transform(valid)
test = testclass.transform(test)

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}
{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3}, 'EDUCATION': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, '4.0': 5, '5.0': 6, '6.0': 7, 'nan': 8}, 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}


In [197]:
cat_info = testclass.category_info

In [198]:
cat_info

{'SEX': {'1.0': 1, '2.0': 2, 'nan': 3},
 'EDUCATION': {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 'MARRIAGE': {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [199]:
Train_X_np = train[in_var].values
Train_y = train[target_var[0]]
Test_X_np = test[in_var].values
Test_y = test[target_var[0]]

In [200]:
print(in_var)

['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


In [201]:
train = train[in_var]

In [202]:
for idx , name in enumerate(in_var) :
    if name in list(cat_info.keys()) :
        cat_info[idx] = cat_info.pop(name)
cat_info

{1: {'1.0': 1, '2.0': 2, 'nan': 3},
 2: {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 3: {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [203]:
sys.getsizeof(pd.get_dummies(train, columns= fac_var))

5280152

In [204]:
sys.getsizeof(train)

4890840

In [205]:
def variable_select(in_var = None , method = None , select_n = None , NTree = None) :
    var_n = len(in_var)
    if method== "sqrt" :
        value =np.sqrt(var_n)
    elif method == "log2" :
        value =np.log2(var_n)
    elif method == "select" :
        value = select_n
    else :
        value = var_n
    return [list(np.random.choice(np.arange(len(in_var)) ,
                                  replace = False , 
                                  size = value)) for _ in range(NTree)]
SELECT_N = 7
select_var = variable_select(in_var=in_var , method="select" , 
                               select_n= 10 ,NTree= SELECT_N)

In [206]:
select_var

[[22, 9, 5, 21, 16, 6, 4, 15, 8, 20],
 [3, 6, 18, 9, 10, 14, 20, 16, 15, 12],
 [16, 2, 18, 9, 12, 6, 20, 10, 17, 4],
 [16, 11, 13, 6, 17, 0, 21, 1, 19, 8],
 [17, 1, 20, 21, 19, 10, 2, 13, 18, 0],
 [7, 0, 18, 5, 21, 22, 14, 20, 17, 6],
 [6, 9, 16, 15, 8, 18, 17, 13, 5, 19]]

In [207]:
import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output

In [208]:
cat_info

{1: {'1.0': 1, '2.0': 2, 'nan': 3},
 2: {'0.0': 1,
  '1.0': 2,
  '2.0': 3,
  '3.0': 4,
  '4.0': 5,
  '5.0': 6,
  '6.0': 7,
  'nan': 8},
 3: {'0.0': 1, '1.0': 2, '2.0': 3, '3.0': 4, 'nan': 5}}

In [209]:
total = np.arange(len(in_var))
fac_idx = list(cat_info.keys())
split = list(set(np.array(fac_idx)) | set(np.array(fac_idx ) + 1))
lists = np.split(total , split)
lists

[array([0]),
 array([1]),
 array([2]),
 array([3]),
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22])]

In [210]:
index_store = []
for idx , i in enumerate(lists) :
    if len(i) == 1 :
        index_store.append([i[0],1])
    else :
        index_store.append([i[0],len(i) ] ) 
index_store

[[0, 1], [1, 1], [2, 1], [3, 1], [4, 19]]

In [211]:
select_var

[[22, 9, 5, 21, 16, 6, 4, 15, 8, 20],
 [3, 6, 18, 9, 10, 14, 20, 16, 15, 12],
 [16, 2, 18, 9, 12, 6, 20, 10, 17, 4],
 [16, 11, 13, 6, 17, 0, 21, 1, 19, 8],
 [17, 1, 20, 21, 19, 10, 2, 13, 18, 0],
 [7, 0, 18, 5, 21, 22, 14, 20, 17, 6],
 [6, 9, 16, 15, 8, 18, 17, 13, 5, 19]]

In [212]:
def tf_mish(x) :
    return x * tf.nn.tanh(tf.nn.softplus(x))
activate_candidate = \
[tf.nn.selu, tf_mish , tf.nn.elu , tf.nn.relu , tf.nn.swish]

In [213]:
class LayerForEnsemble(object) :
    def __init__(self , fac_var,category_info) : 
        self.fac_var = fac_var
        self.info = category_info
    def run(self, X , select_var, name, type="embedding", emb_dim = 4) :
        inputs = []
        for idx in select_var :
            split = tf.slice(X, [0, idx], [-1, 1])
            if idx in list(self.info.keys()) :
                split =tf.cast(split, dtype=tf.int32)
                split = tf.reshape(split,(-1,))
                size = max(list(self.info[idx].values())) + 1
                if type == "embedding" :    
                    Cat = tf.keras.layers.Embedding(size,emb_dim)(split)
                elif type == "onehot" :
                    Cat = tf.one_hot(split ,depth=size)
                else :
                    raise Exception(f"No Valid Type : {type}, Please Change the type to onehot or embedding")
                inputs.append(Cat)
            else :
                inputs.append(split)
        x_input = tf.concat(inputs , axis = 1, name=name)
        return x_input
def Layer(X , hdims, name , activation,
          batch_prob,DropoutRate,reuse = False) :
    with tf.variable_scope(f"EnsLayer_{name}",reuse=reuse):
        for idx2 , h_dim in enumerate(hdims) :
            if idx2 == 0 :
                W = tf.get_variable(f"w_{idx2}",shape=[X.get_shape()[1],h_dim])
                B = tf.get_variable(f"b_{idx2}",shape=[h_dim])
                layer = tf.matmul(X, W) + B
            else :
                W = tf.get_variable(f"w_{idx2}",shape=[hdims[idx2-1] ,h_dim ])
                B = tf.get_variable(f"b_{idx2}",shape=[h_dim])    
                layer = tf.matmul(layer, W) + B
            if len(hdims) == idx2 + 1 :
                logit = layer
            else :
                layer = tf.layers.\
                batch_normalization(layer, center=True, 
                                    scale=True, training=batch_prob)
                layer = activation(layer)
                layer = tf.nn.dropout(layer, keep_prob=DropoutRate)
    return logit
    
def EnsembleNN(X , hidden = [[],[]], batch_prob=None,
               DropoutRate=None, Combs=None) :
    Ensembles = []
    RepLayer = LayerForEnsemble(fac_var , cat_info)
    with tf.variable_scope("NNEnsemble") :
        for idx , Comb in enumerate(Combs) :
            x_input = RepLayer.run(X, Comb, f"nnTree{idx}","onehot")
            X_DIM = x_input.get_shape().as_list()[1]
            dims = hidden[idx]
            dims = [X_DIM] + dims
            print(f"No.{idx} nnTree : {str(dims)}")
            SELECT = np.random.randint(0 , len(activate_candidate) , 1)[0]
            activation = activate_candidate[SELECT]
            LAYER = Layer(x_input , dims, idx , activation,batch_prob,DropoutRate)
            Ensembles.append(LAYER)
    return Ensembles

In [214]:
row , dim = Train_X_np.shape
print(row,dim)
target_n = 2 
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [ None , dim])
y = tf.placeholder(tf.float32, shape = [ None , 1])
DropoutRate = tf.placeholder(tf.float32, name ="dropoutRate")
training = tf.placeholder_with_default(False, shape=[], name="training")
global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
batch_prob  = tf.placeholder(tf.bool)
batch_size = tf.placeholder(tf.int64)

30000 23


In [215]:
data_tuple = (X,y)
class_dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
class_dataset = class_dataset.shuffle(buffer_size= 30000,)
class_dataset = class_dataset.batch(batch_size=batch_size, drop_remainder=True)
iter = class_dataset.make_initializable_iterator()
feature_x , label_y = iter.get_next()

In [216]:
len(select_var)

7

In [217]:
HIDDEN = [
    [ 40 , 40, 15, 1] , 
    [ 30 , 10, 1] ,
    [ 40 , 30, 15, 1] ,
    [ 30 , 30, 15, 1] ,
    [ 30 , 25, 16, 1] ,
    [ 30, 15, 1] , 
    [ 20 , 10, 1]
]

NModels = EnsembleNN(feature_x , hidden = HIDDEN , 
                     batch_prob=batch_prob, DropoutRate=DropoutRate,
                     Combs= select_var )

No.0 nnTree : [10, 40, 40, 15, 1]
No.1 nnTree : [15, 30, 10, 1]
No.2 nnTree : [18, 40, 30, 15, 1]
No.3 nnTree : [13, 30, 30, 15, 1]
No.4 nnTree : [21, 30, 25, 16, 1]
No.5 nnTree : [10, 30, 15, 1]
No.6 nnTree : [10, 20, 10, 1]


In [218]:
weights_control = tf.get_variable("weight_control__",shape = [len(NModels)] , 
                                  dtype = tf.float32 , 
                                  initializer = tf.constant_initializer(1/len(NModels)),
                                 constraint= lambda x: tf.clip_by_value(x, 0, 0.4))

In [219]:
weights_control = tf.nn.softmax(weights_control)
# weights_tree_trans = tf.reshape(weights_control,(len(NModels),1,1))
#weighted_logit = weights_control * tf.concat(NModels,axis=1)
Probs = []
Losses = []
argMax = [ ]
for idx , Model in enumerate(NModels) :
    loss2 = tf.nn.\
    weighted_cross_entropy_with_logits(
        labels = label_y , logits=Model,pos_weight=1.5)
    PROB = tf.nn.sigmoid(Model)
    argmax = tf.where(PROB > tf.constant(0.5), 
                      tf.ones_like(PROB), tf.zeros_like(PROB))
    argMax.append(argmax)
    Probs.append(PROB)
    Losses.append(loss2)
#     Probs += tf.nn.sigmoid(Model)
prob = tf.reshape(tf.reduce_sum(tf.concat(Probs,axis=1)*weights_control,axis=1),(-1,1))
total_argmax = tf.where(prob > tf.constant(0.5), 
                      tf.ones_like(prob), tf.zeros_like(prob))
argMax.append(total_argmax)

In [220]:
argMax.append(label_y)

In [221]:
comp_pred = tf.equal( total_argmax, label_y )
accuracy = tf.reduce_mean(tf.cast(comp_pred, tf.float32))

In [222]:
loss2 = tf.reduce_mean(tf.reduce_sum(tf.concat(Losses,axis=1)*weights_control,axis=1))

In [223]:
# Logit = tf.concat([Logit], axis = 0) * weights_tree_trans
# Logits = tf.reduce_sum(Logit, axis = 0)
# Probs = Probs / len(NModels)
# weighted_mean_logit = tf.reshape(tf.reduce_mean(weighted_logit,axis=1),(-1,1))
# prob= tf.nn.sigmoid(weighted_mean_logit)

In [224]:

auc_value , update_auc = tf.metrics.auc(label_y , prob , curve="ROC")

In [225]:
#class_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "NNEnsemble") 
totalvars = tf.trainable_variables()

In [226]:
learning_rate = tf.train.cosine_decay_restarts(1e-4, global_step,
                                               first_decay_steps=100, t_mul=1.5,m_mul=0.9, alpha=0.0)

L2= []
for v in totalvars :
    L2.append(tf.nn.l2_loss(v))
L2Regularizer= tf.add_n(L2)  * 0.01
loss2 +=L2Regularizer
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    solver2 = tf.train.AdamOptimizer(learning_rate= learning_rate).minimize(loss2 ,var_list = totalvars)

In [227]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, TransformedBbox,
                                                   BboxPatch, BboxConnector)
def my_mark_inset(parent_axes, inset_axes, loc1a=1, loc1b=1, loc2a=2, loc2b=2, **kwargs):
    rect = TransformedBbox(inset_axes.viewLim, parent_axes.transData)
    pp = BboxPatch(rect, fill=False, **kwargs)
    parent_axes.add_patch(pp)
    p1 = BboxConnector(inset_axes.bbox, rect, loc1=loc1a, loc2=loc1b, **kwargs)
    inset_axes.add_patch(p1)
    p1.set_clip_on(False)
    p2 = BboxConnector(inset_axes.bbox, rect, loc1=loc2a, loc2=loc2b, **kwargs)
    inset_axes.add_patch(p2)
    p2.set_clip_on(False)
    return pp, p1, p2
def subplotting(ax , store , x , y ,cond={}, **kwargs) :
    ax.plot(store[x],store[y],**kwargs)
    if "ylabel" in cond :
        ax.set_ylabel(cond["ylabel"], fontsize= 10)
    if "xlabel" in cond :
        ax.set_xlabel(cond["xlabel"], fontsize= 10)
    if "xlim" in cond :
        ax.set_xlim(cond["xlim"])
    if "ylim" in cond :
        ax.set_ylim(cond["ylim"])
    if "title" in cond :
        ax.set_title(cond["title"], fontsize= 15)
    ax.legend()
    return ax
## version3
def vis_onlysl(store:dict, path:str,title:str) :
    clear_output()
    fig = plt.figure(figsize=(10, 10))
    gs = GridSpec(nrows=2, ncols=2)
    ax1 = fig.add_subplot(gs[0:1, 0])
    ax2 = fig.add_subplot(gs[0:1, 1])
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, 
            top=0.9, wspace=0.3, hspace=0.2)
    ax1.plot(store["epoch"],store["slloss"],label = "SLloss", color='c')
    ax1.set_xlabel('Epoch')
    select_sl= np.argmin(store["slloss"])
    ax1.vlines(store["epoch"][select_sl],
               np.min(store["slloss"]),np.max(store["slloss"]),
               label='Best', color='c')
    a =store["slloss"][-1]
    ax1.set_title(f'Loss : {a:.4f}')
    ax1.set_ylabel('Supervised', fontsize= 15)
    ax1.legend()
    ax2.plot(store["epoch"],store["auc"],label = "Train auc")
    ax2.plot(store["epoch"],store["teauc"],label = "Test auc")
    ax2.set_ylabel('AUC', fontsize= 15)
    ax2.set_xlabel('Epoch')
    select= np.argmax(store["teauc"])
    msg = f"test idx : {store['epoch'][select]}, maximum : {store['teauc'][select]*100:.2f}"
    ax2.set_title(msg)             
#     ax2.set_ylim(0.7,store['auc'][select]+0.1)
    ax2.legend()
    check_n = 4
    if len(store['epoch']) > check_n :
        axins = inset_axes(ax2, "100%", "100%", 
                           bbox_to_anchor=[0.36, .3, .5, .4],
                       bbox_transform=ax2.transAxes, borderpad=0)
        axins.plot(store['epoch'], store['auc'])
        axins.plot(store['epoch'], store['teauc'])
        maximum = np.max(store['auc'][-check_n:] + store['teauc'][-check_n:])
        minumum = np.min(store['auc'][-check_n:] + store['teauc'][-check_n:])
        xlims = (store['epoch'][-check_n],store['epoch'][-1])
        ylims = (minumum, maximum)
        axins.set(xlim=xlims, ylim=ylims)
        my_mark_inset(ax2, axins, loc1a=2, loc1b=3, loc2a=4, loc2b=4, fc="none", ec="0.5") # 
#     msg = f"Epoch : {epoch[-1]}, Loss : {loss[-1]:.3f}, Auc : {aucs[-1]:.3f}"
#     skplt.metrics.plot_ks_statistic(store["train_y"], store["train_prob"], 
#                                 ax = ax3 ,
#                                 title = "[Train] KS Static PLOT")
#     skplt.metrics.plot_ks_statistic(store["test_y"], store["test_prob"], 
#                                 ax = ax4 ,
#                                 title = "[Test] KS Static PLOT")
    sns.boxplot(x="t", y="prob", data=store["train_pd"] , ax = ax3)
    ax3.set_title("train" , fontsize= 15)
    sns.boxplot(x="t", y="prob", data=store["test_pd"] , ax = ax4)
    ax4.set_title("test" , fontsize= 15)
    plt.suptitle(title)
    plt.savefig(path)
    plt.show()

In [ ]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
Epoch = 100000
oversample = list(np.arange(len(Train_y))) + 2 * list(np.where(Train_y.values == 1)[0])
from matplotlib.gridspec import GridSpec
from sklearn.metrics import auc , roc_auc_score
with tf.Session(config=config) as sess:
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init)
    store = {"epoch" : [] ,  "slloss" : [],
             "auc" : [], "teauc" : []}
    store["train_pd"] = []
    store["test_pd"] = []
    for i in range(Epoch):    
        batchSlLoss = []
        permutation_idx = np.random.permutation(oversample)
        sess.run(iter.initializer , 
                 feed_dict= {X : Train_X_np[permutation_idx],
                             y : Train_y.values.reshape(-1,1)[[permutation_idx]],
                             batch_size : 300,
                            }) # switch to train dataset
        while True :
            try :
                ### Supervised Learning
                result  = sess.run([solver2 , loss2],
                                   feed_dict={global_step:i, 
                                              batch_prob :True,
                                              DropoutRate : 0.8
                                             })
                batchSlLoss.append(result[1])
            except tf.errors.OutOfRangeError :
                break
        mslloss = np.mean(batchSlLoss)
        print(f"Epoch : {i}, SLLoss : {mslloss:.6f}" , end = "\r")
        if (i % 100 == 0) :
            sess.run(iter.initializer , 
                     feed_dict= {X : Train_X_np,
                     y : Train_y.values.reshape(-1,1),
                     batch_size : len(Train_X_np),}) 
            tr_result  = sess.run([prob,label_y],
                                  feed_dict={batch_prob :False,
                                             DropoutRate : 1.0
                                            })
            sess.run(iter.initializer , 
                     feed_dict= {X : Test_X_np,
                     y : Test_y.values.reshape(-1,1),
                     batch_size : len(Test_X_np),}) 
            result  = sess.run([prob,label_y,weights_control],
                               feed_dict={batch_prob :False,
                                          DropoutRate : 1.0
                                         })
            plt.scatter(np.arange(0,len(result[2])), result[2])
            plt.savefig(f"{weightResultPath}/plot.{i:05d}.png")
            a = "Weight nnTree" +" \n"
            b = a + str(result[2])
            plt.title(b)
            plt.close()
            sess.run(iter.initializer , 
                     feed_dict= {X : Test_X_np,
                     y : Test_y.values.reshape(-1,1),
                     batch_size : len(Test_X_np),}) 
            dd  = sess.run(argMax+[accuracy],
                               feed_dict={batch_prob :False,
                                          DropoutRate : 1.0
                                         })
            s = pd.DataFrame(np.concatenate(dd[:-1],axis=1)).T
            models = np.arange(s.shape[0]-2).tolist()
            models = models + ["pred","real"]
            indexs = np.arange(s.shape[1])
            fig, ax = plt.subplots( figsize = (12,8))
            plt.pcolor(s)
            plt.yticks(np.arange(0.5, len(models), 1),models)
            plt.title(f"Epoch {i} , Auccray : {dd[-1]*100:.2f}",fontsize=20)
            plt.savefig(f"{testPath}/plot.{i:05d}.png")
            plt.close()  
            store["epoch"].append(i)
            store["slloss"].append(mslloss)
            store["train_pd"] = pd.DataFrame(np.concatenate(tr_result,axis=1),
                                             columns=["prob","t"])
            store["test_pd"] = pd.DataFrame(np.concatenate(result[0:2],axis=1),
                                             columns=["prob","t"])
            store["auc"].append(roc_auc_score(store["train_pd"]["t"].values,
                                              store["train_pd"]["prob"].values))
            store["teauc"].append(roc_auc_score(store["test_pd"]["t"].values,
                                                store["test_pd"]["prob"].values))
#             store["train_prob"] =  np.concatenate((1-tr_result[2],tr_result[2]),axis=1)
#             store["test_prob"] =  np.concatenate((1-result[2],result[2]),axis=1)
            vis_onlysl(store, f"{ResultPath}/0405_SL_EnsembelNN.png",
                       title="Train Ensemblem Neural Network")
            clear_output()

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1



In [136]:
len(dd[:-1])

9

In [ ]:
for key, value in  category_info.items() :
    a[key] = pd.Categorical(a[key],list(value.values()))
pd.get_dummies(a,columns=fac_var).columnsumns.tolist()

In [86]:
pd.get_dummies(a)

,SEX_1,SEX_2,SEX_3,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,EDUCATION_7,EDUCATION_8,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,MARRIAGE_4,MARRIAGE_5
